In [6]:
pip install llama-index-llms-gemini llama-index-llms-groq


  Obtaining dependency information for llama-index-llms-gemini from https://files.pythonhosted.org/packages/ff/5c/cee95dc6442d91dbc7bb185c8dbc00fb3afea6e814bd20e8b4f323d89b1e/llama_index_llms_gemini-0.5.0-py3-none-any.whl.metadata
  Using cached llama_index_llms_gemini-0.5.0-py3-none-any.whl.metadata (3.3 kB)
  Obtaining dependency information for llama-index-llms-groq from https://files.pythonhosted.org/packages/67/70/110b5098829737abea1a242e525b05bdbeaa448f6f5b46b08b760c6b22d4/llama_index_llms_groq-0.3.2-py3-none-any.whl.metadata
  Using cached llama_index_llms_groq-0.3.2-py3-none-any.whl.metadata (2.0 kB)
  Obtaining dependency information for google-generativeai>=0.5.2 from https://files.pythonhosted.org/packages/6e/40/c42ff9ded9f09ec9392879a8e6538a00b2dc185e834a3392917626255419/google_generativeai-0.8.5-py3-none-any.whl.metadata
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Obtaining dependency information for pillow<11,>=10.2.0 from https://files.p

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Admin\\OneDrive\\Documents\\Repos\\note-to-knowledge\\venv\\Lib\\site-packages\\~il\\_imaging.cp311-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install llama-index-llms-google-genai llama-index

  Obtaining dependency information for llama-index-llms-google-genai from https://files.pythonhosted.org/packages/07/a2/73fb6c15b7e84eaf18f7cecefca97766a778d299e407893cc60e34314d1a/llama_index_llms_google_genai-0.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/93/bf/a3e72da688b24c0a89919c8cc7cd62bbea2aeb6807322a8025743c7e809b/llama_index-0.12.41-py3-none-any.whl.metadata
  Using cached llama_index-0.12.41-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for google-genai>=1.4.0 from https://files.pythonhosted.org/packages/c4/ae/64fccdebf5811453ce53b0d5ee23d4f27ef173ef36d3b67dad791a0007aa/google_genai-1.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-agent-openai<0.5,>=0.4.0 from https://files.pythonhosted.org/packages/ec/3e/24fcc3e486c730e4f0bbd7bd94f22adce2e6f570689b8fa1f0eddf7ad71b/llama_index_agent_openai-0.4.9-py3-none-any.whl.metadata
  Using cached ll

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\Admin\\OneDrive\\Documents\\Repos\\note-to-knowledge\\venv\\Lib\\site-packages\\llama_cloud\\types\\hugging_face_inference_api_embedding.py'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from dotenv import load_dotenv
from llama_index.core import Document
from llama_index.llms.openai import OpenAI
from llama_index.core.graph_stores.simple import SimpleGraphStore
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# 1. Load environment variables for API keys
load_dotenv()
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [2]:
# 2. Prepare your notes
notes = [
    "Marie Curie discovered radium and polonium. She won the Nobel Prize in Physics.",
    "Albert Einstein developed the theory of relativity."
]
documents = [Document(text=note) for note in notes]


In [7]:
# 3. Choose your LLM backend

# --- Use Gemini ---
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.openai import OpenAIEmbedding
llm = Gemini(api_key=GEMINI_API_KEY, model="gemini-1.5-flash")  # or "gemini-pro", etc.

# ---- Use OPenai ----
# llm = OpenAI(model="gpt-4-turbo")  # You can use "gpt-3.5-turbo" for lower cost
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# --- Use Groq (uncomment to use Groq instead) ---
# from llama_index.llms.groq import Groq
# llm = Groq(api_key=GROQ_API_KEY, model="mixtral-8x7b-32768")  # or "llama-3-70b-8192", etc.

C:\Users\Admin\AppData\Local\Temp\ipykernel_11016\3865313772.py:6: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(api_key=GEMINI_API_KEY, model="gemini-1.5-flash")  # or "gemini-pro", etc.


In [11]:
from llama_index.core.indices.property_graph import PropertyGraphIndex, SimpleLLMPathExtractor
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine


# 3. Set up the triplet extractor
kg_extractor = SimpleLLMPathExtractor(llm=llm)

# 4. Build the property graph index (your knowledge graph)
kg_index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[kg_extractor],
    embed_model=embed_model
)


In [12]:
# 5. Set up the Knowledge Graph RAG Retriever and Query Engine
kg_retriever = KnowledgeGraphRAGRetriever(storage_context=kg_index.storage_context)
query_engine = RetrieverQueryEngine.from_args(kg_retriever, llm=llm)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11016\1898775639.py:2: DeprecationWarning: Call to deprecated class KnowledgeGraphRAGRetriever. (KnowledgeGraphRAGRetriever is deprecated, it is recommended to use PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  kg_retriever = KnowledgeGraphRAGRetriever(storage_context=kg_index.storage_context)


In [ ]:

# 6. Query your GraphRAG system!
question = "Who discovered radium?"
response = query_engine.query(question)
print("Answer:", response.response)

# Optional: Inspect your knowledge graph
print("Nodes:", kg_index.)
print("Relationships:", kg_index.get_all_relationships())

Answer: Empty Response


AttributeError: 'PropertyGraphIndex' object has no attribute 'get_all_nodes'